Black-Scholes-Merton

In [1]:
# 옵션 값은 opstrat 의 black_scholes() 함수를 사용하여 계산
# Opstrat 옵션을 다루는 파이썬 패키지
# 모델에 필요한 모든 입력 은 인수로 전달되어야 함.
# 위의 입력 외에도 옵션의 유형은 유형 매개변수를 사용하여 지정해야 함(콜 경우 'c ' , 풋 경우 'p'

In [2]:
pip install opstrat

In [3]:
# 라이브러리 호출
# 필요 값 입력
import opstrat as op
#Declare parameters
K=200    #spot price
St=208   #current stock price
r=4      #4% risk free rate
t=30     #time to expiry, 30 days 
v=20     #volatility 
type='c' #Option type call

#Black Scholes Model
bsm=op.black_scholes(K=K, St=St, r=r, t=t, 
                     v=v, type='c')

In [4]:
# 옵션가격(프리미엄)
# 프리미엄 = 내재가치 + 시간가치
bsm['value']

{'option value': 10.210518559926442,
 'intrinsic value': 8,
 'time value': 2.2105185599264416}

In [5]:
# 옵션 민감도
bsm['greeks']

{'delta': 0.7793593241701937,
 'gamma': 0.024868265088898882,
 'theta': -0.07559961986526405,
 'vega': 0.17686037602292404,
 'rho': 0.12484620893217029}

In [6]:
#  옵션가격  수학적으로 계산하기

In [7]:
import math as m # math 모듈 임포트

In [8]:
S = 100
K = 100
T = 5
V = 0.3
Rf = m.log(1.05)
print('연속복리무위험이자율 =', Rf)
print('연속복리를 적용한 1년뒤 종가 =', m.exp(Rf))

연속복리무위험이자율 = 0.04879016416943204
연속복리를 적용한 1년뒤 종가 = 1.05


In [9]:
numerator = m.log(S/K) + (Rf + 1/2 * V**2) * T
denominator = V * m.sqrt(T)
d1 = numerator / denominator
d2 = d1 - V * m.sqrt(T)
print('d1 =', d1, ',', 'd2 =', d2)

d1 = 0.6990706090123837 , d2 = 0.028250215762446795


In [10]:
numerator = m.log(S/K) + (Rf + 1/2 * V**2) * T
denominator = V * m.sqrt(T)
d1 = numerator / denominator
d2 = d1 - V * m.sqrt(T)
print('d1 =', d1, ',', 'd2 =', d2)

d1 = 0.6990706090123837 , d2 = 0.028250215762446795


In [11]:
import scipy as sp # scipy 모듈 임포트
import scipy.stats # scipy에서 통계 모듈 임포트
normdis = sp.stats.norm() # 변수에 표준정규분포를 입력해줌 

In [12]:
N_d1 = normdis.cdf(d1) # N(d1) 산출
N_d2 = normdis.cdf(d2) # N(d2) 산출
print('N(d1) =', N_d1, ',', 'N(d2) =', d2)

N(d1) = 0.7577460474065406 , N(d2) = 0.028250215762446795


In [13]:
# 이항모형(CRR 모형) 옵션가격
# https://pythoncpa.tistory.com/2?category=806912  내용 참고

In [14]:
import math as m

In [15]:
S0 = 100
V = 0.3
T = 5   # 이항분포 전체기간
dt = 1  # 이항분포가 1회발생하는 단위기간, 연간 변동성 기준으로 월 1회 발생하는 경우 1/12= 0.833
Rf = 0.05
K = 100

In [16]:
u = m.exp(V * m.sqrt(dt))
d = 1 / u
P = (m.exp(Rf * dt) - d) / (u - d)
print('u =', u)
print('d =', d)
print('P =', P)

u = 1.3498588075760032
d = 0.7408182206817179
P = 0.5097408651817704


In [21]:
import numpy as np
import pandas as pd
S_tree = pd.DataFrame(np.zeros((T + 1) * (T + 1)).reshape(T + 1, T + 1))
S_tree

,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
for node in range(T + 1):
      for time in range(T + 1):
         if time >= node:
           S_tree.loc[node, time] = S0 * m.pow(u, (time - node)) * m.pow(d, node)
S_tree

,0,1,2,3,4,5
0,100.0,134.985881,182.211880,245.960311,332.011692,448.168907
1,0.0,74.081822,100.000000,134.985881,182.211880,245.960311
2,0.0,0.000000,54.881164,74.081822,100.000000,134.985881
3,0.0,0.000000,0.000000,40.656966,54.881164,74.081822
4,0.0,0.000000,0.000000,0.000000,30.119421,40.656966
5,0.0,0.000000,0.000000,0.000000,0.000000,22.313016


In [24]:
C_K_tree = pd.DataFrame(np.zeros((T + 1) * (T + 1)).reshape(T + 1, T + 1))
for node in range(T + 1):
  for time in range(T + 1):
    if time >= node:
      C_K_tree.loc[node, time] = max(S_tree.loc[node, time] - K, 0)
C_K_tree

,0,1,2,3,4,5
0,0.0,34.985881,82.21188,145.960311,2.320117e+02,348.168907
1,0.0,0.000000,0.00000,34.985881,8.221188e+01,145.960311
2,0.0,0.000000,0.00000,0.000000,2.842171e-14,34.985881
3,0.0,0.000000,0.00000,0.000000,0.000000e+00,0.000000
4,0.0,0.000000,0.00000,0.000000,0.000000e+00,0.000000
5,0.0,0.000000,0.00000,0.000000,0.000000e+00,0.000000


In [30]:
for node in range(T - 1, -1, -1):
      for time in range(T - 1, -1, -1):
         if time >= node:
             C_K_tree.loc[node, time] = max(
               P * C_K_tree.loc[node, time + 1] / (1 + Rf) + (1 - P) * C_K_tree.loc[node + 1, time + 1] / (1 + Rf),
               C_K_tree.loc[node, time]
      )
C_K_tree

,0,1,2,3,4,5
0,37.042002,61.314001,99.128897,155.853227,237.175520,348.168907
1,0.000000,15.583290,28.249654,50.260325,87.194365,145.960311
2,0.000000,0.000000,4.002886,8.245426,16.984508,34.985881
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [34]:
# plotly  활용 옵션시각화
#  추후 작업 업로드
# https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=stochastic73&logNo=221553746353  참고
## 퀀트립 (QuantLib) 활용 
# https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=stochastic73&logNo=22174610

In [35]:
pip install QuantLib-Python

     --------------------------------------- 12.5/12.5 MB 17.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
